In [11]:
import pandas as pd
import numpy as np
import json
import shapely.geometry

In [31]:
with open('../data/Official_Neighborhoods.geojson') as file:
  data = json.load(file)
features = data['features']
neighborhoods = pd.DataFrame()
for feature in features:
  geo_type = feature['geometry']['type']
  if geo_type == 'Polygon':
    geometry = shapely.geometry.Polygon(feature['geometry']['coordinates'][0])
  elif geo_type == 'MultiPolygon':
    geometry = shapely.geometry.MultiPolygon(feature['geometry']['coordinates'])
  else:
    geometry = np.nan
  neighborhood = pd.DataFrame({
    'id': feature['properties']['OBJECTID'],
    'name': feature['properties']['NAME'],
    'geo_type': feature['geometry']['type'],
    'geometry': geometry
  }, index=[0])
  neighborhoods = pd.concat([neighborhoods, neighborhood], ignore_index=True)
  
with open('../data/zillow_all_active_atlanta.csv') as file:
  active_df = pd.read_csv(file, header=0)

with open('../data/zillow_sold_atlanta.csv') as file:
  sold_df = pd.read_csv(file, header=0)
  
df = pd.concat([active_df, sold_df], ignore_index=True)
df.reset_index(drop=True, inplace=True)

def find_neighborhood_id(lng, lat):
  point = shapely.geometry.Point(lng, lat)
  in_neighborhood = neighborhoods.apply(lambda row: row['geometry'].contains(point), axis=1)
  indices = in_neighborhood[in_neighborhood].index
  if len(indices) == 0:
    return -1
  return neighborhoods.loc[indices[0], 'id']

df['neighborhood_id'] = df.apply(lambda row: find_neighborhood_id(row['longitude'], row['latitude']), axis=1)

df.to_csv('../data/zillow_atlanta_with_neighborhood.csv', index=False)

/var/folders/6v/n3x50m4s03380xg5w06p_2nw0000gn/T/ipykernel_52831/2590784771.py:25: DtypeWarning: Columns (37,66,76,81,121) have mixed types. Specify dtype option on import or set low_memory=False.
  sold_df = pd.read_csv(file, header=0)
